In [1]:
# Import the QICK drivers and auxiliary libraries
from qick import *
from qick.parser import load_program
from sequence import sequence_TTL
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load bitstream with custom overlay
soc = QickSoc()

soccfg = soc
print(soccfg)
out_chs = [0,1]


QICK configuration:

	Board: RFSoC4x2

	Global clocks (MHz): tProcessor 350.000, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 0, envelope memory 65536 samples
		DAC tile 0, ch 0, 32-bit DDS, fabric=614.400 MHz, fs=9830.400 MHz
	1:	axis_signal_gen_v6 - tProc output 1, envelope memory 65536 samples
		DAC tile 2, ch 0, 32-bit DDS, fabric=614.400 MHz, fs=9830.400 MHz

	2 readout channels:
	0:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, ch 0, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 4, tProc input 0
	1:	axis_readout_v2 - controlled by PYNQ
		ADC tile 0, ch 1, 32-bit DDS, fabric=491.520 MHz, fs=3932.160 MHz
		maxlen 1024 (avg) 16384 (decimated), trigger bit 5, tProc input 1

	2 DACs:
		DAC tile 0, ch 0 is DAC_B
		DAC tile 2, ch 0 is DAC_A

	2 ADCs:
		ADC tile 0, ch 0 is ADC_D
		ADC tile 0, ch 1 is ADC_C

	4 digital output pins (tProc output 7):
	0:	PMOD0_0
	1:	PMOD0_1
	2:	PMOD0_2
	3

In [3]:
def divide_nested_list(nested_list, divisor):
    result = []
    for sublist in nested_list:
        divided_sublist = []
        for number in sublist:
            divided_sublist.append(round(number / divisor))
        result.append(divided_sublist)
    return result

In [11]:
class MultiSequenceProgram(AveragerProgram):
    def __init__(self,soccfg, cfg):
        super().__init__(soccfg, cfg)

    def initialize(self):
        cfg=self.cfg 
        
        #EOM
        for ch in out_chs: #[0,1]
            #READOUT AT CHANNEL 0 and 1
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"])

        idata = 30000*np.ones(16*cfg["EOM"]["length"])
#         qdata = 30000*np.ones(16*cfg["length"])

        for ch in self.cfg["EOM"]['out_ch']:
            #GENERATE AT CHANNEL 0 and 1
            self.declare_gen(ch=ch, nqz=cfg["EOM"]["zone"])
            #ADD PULSE AT CHANNEL 0 and 1
            self.add_pulse(ch=ch, name="measure", idata=idata)
#             self.add_pulse(ch=ch, name="measure", idata=idata,qdata=qdata)
        
        freq=soccfg.freq2reg(cfg["pulse_freq"])  # convert frequency to dac frequency
#         self.trigger(pins=[0], t=0) # send a pulse on pmod0_0, for scope trigger
        for ii, ch in enumerate(self.cfg["EOM"]['out_ch']):
            #PULSE REGISTER AT CHANNEL 0 and 1
            self.default_pulse_registers(ch=ch,style="arb",waveform="measure", mode=cfg["EOM"]["mode"])
                
    def body(self):
        cfg=self.cfg 
        
        #EOM
        self.trigger(adcs=[0,1],adc_trig_offset=cfg["adc_trig_offset"])  # trigger the adc acquisition
        for ii, ch in enumerate(cfg["EOM"]['out_ch']):
            for jj,freq in enumerate(cfg["EOM"]["freq_seq"]):
                #PULSE REGISTER AT CHANNEL 0 and 1
                self.set_pulse_registers(ch=ch, freq=soccfg.freq2reg(cfg["EOM"][freq]["pulse_freq"]), phase=soccfg.deg2reg(cfg["EOM"][freq]["res_phase"]), gain=cfg["EOM"][freq]["pulse_gain"])
                #PULSE AT CHANNEL 0 and 1
                self.pulse(ch=ch, t=cfg["EOM"]["time_seq"][jj]) # play readout pulse
        
        #AOM
        self.cfg["AOM"]["time"] = divide_nested_list(self.cfg["AOM"]["time"],2.6)
        self.cfg["AOM"]["length"] = divide_nested_list(self.cfg["AOM"]["length"],2.6)
        
        time_list,length,pins_seq = sequence_TTL(self.cfg["AOM"]["time"], self.cfg["AOM"]["length"], self.cfg["AOM"]["pins"])
        for time_index, time in enumerate(time_list):
            self.trigger(adcs=self.ro_chs,
                    pins=pins_seq[time_index], 
                    adc_trig_offset=self.cfg["adc_trig_offset"],
                     t = time,
                    width = length[time_index])
        self.wait_all()
        self.sync_all(self.us2cycles(self.cfg["relax_delay"]))
        


#Global Config
#EOM
freqA = {"res_phase": 90 , # The phase of the signal        

        "pulse_gain": 2000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": soccfg.adcfreq(1000, gen_ch=0, ro_ch=0), # [MHz]
        }
freqB = {"res_phase": 180 , # The phase of the signal        

        "pulse_gain": 3000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units
        "pulse_style": "const", # --Fixed
        "pulse_freq": soccfg.adcfreq(1000, gen_ch=0, ro_ch=0), # [MHz]
        }
A = "freqA"
B = "freqB"
configEOM={"out_ch":out_chs,
        # "reps":1, # --Fixed
        "freqA": freqA,
        "freqB": freqB,
        "freq_seq": [A,B,A,B],
#         "res_phase": [180 ,180],
#         "pulse_gain":[32000,32000],
        "time_seq":[50, 190,370,700],
        # pulse length: will be repeated periodically
        "length":100, # [Clock ticks]
        "pulse_freq":soccfg.adcfreq(1000, gen_ch=0, ro_ch=0), #readout freq
        # "readout_length": 2000, # [Clock ticks]
        
        "zone": 1,  
        # nyquist zone of generator
        
        "mode": "periodic",
        # to make CW output
        
        #"mode": "oneshot"
        # to make single shot output

       }

#TTL
configAOM={
        "length":[[200,200,200,200],[200,200,200,200],[200,200,200,200],[200,200,200,200]], # [Clock ticks]
        "pins":[1,2,0,3],
        "time":[[0,400,800,1200],[0,400,800,1200],[0,400,800,1200],[0,400,800,1200]],
        
        # Try varying soft_avgs from 1 to 200 averages
       }
config= {
    
    "adc_trig_offset": 100, # [Clock ticks]
    "soft_avgs":1,
    # "zone": 1,  
    "relax_delay":1.0, # --us
    #Readout
    "readout_length": 2000, # [Clock ticks]
    "pulse_freq":soccfg.adcfreq(1000, gen_ch=0, ro_ch=0),
    "reps":1, # --Fixed
    "EOM":configEOM,
    "AOM":configAOM
    }




In [12]:
prog = MultiSequenceProgram(soccfg, config)
import time
while True:
    iq_list = prog.acquire_decimated(soc)
    time.sleep(2)

# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
#     plt.plot(iq[0], label="I value, ADC %d"%(config['out_ch'][ii]))
#     plt.plot(iq[1], label="Q value, ADC %d"%(config['out_ch'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config["EOM"]['out_ch'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()


# to switch off output
soc.reset_gens()

Readout time 0 appears to conflict with previous readout ending at 1524.153646?
Readout time 0 appears to conflict with previous readout ending at 1524.153646?
Readout time 154 appears to conflict with previous readout ending at 1524.153646?
Readout time 154 appears to conflict with previous readout ending at 1524.153646?
Readout time 308 appears to conflict with previous readout ending at 1678.153646?
Readout time 308 appears to conflict with previous readout ending at 1678.153646?
Readout time 462 appears to conflict with previous readout ending at 1832.153646?
Readout time 462 appears to conflict with previous readout ending at 1832.153646?


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
#EOM

f0_start=1003
f0_step=0.000250
expts=40
# expts=1

f0_v = np.arange(0,expts)*f0_step+f0_start

f0_v = soccfg.adcfreq(f0_v, gen_ch=0, ro_ch=0)

def calculate_phase(d):
    [xi,xq] = d
    x = xi +1j*xq

    # Average to improve calibration.
    xavg = np.mean(x)

    # Calculate calibration phase.
    fi = np.remainder(np.angle(xavg,deg=True)+360,360)
    return [fi, np.abs(xavg), np.std(x)]

def print_results(f0, res):
    print("freq_i = %f MHz, \n"%(f0) +
         "phi_i = (%.2f, %.2f) deg, \n" % tuple([res[i][0] for i in range(2)]) +
         "mag = (%.2f, %.2f), \n" % tuple([res[i][1] for i in range(2)]) +
         "RMS = (%.2f, %.2f) ADU" % tuple([res[i][2] for i in range(2)]))
#     print("freq_i = %f MHz, phi_i = (%.2f, %.2f) deg, mag = (%.2f, %.2f), RMS = (%.2f, %.2f) ADU" %(f0,*out_array,*A,*xrms))

prog = MultipleFreqProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)

# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
#     plt.plot(iq[0], label="I value, ADC %d"%(config['out_ch'][ii]))
#     plt.plot(iq[1], label="Q value, ADC %d"%(config['out_ch'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['out_ch'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()


# to switch off output
soc.reset_gens()


# get the phase
res = [calculate_phase(iq) for iq in iq_list]
print_results(config['pulse_freq'], res)

p_diff = res[0][0]-res[1][0]
print('Phase difference of {:} deg'.format(p_diff))

# if config['res_phase'] == [0, 0]:
#     p_diff0 = p_diff.copy() 

